RNN-LSTM Approach 3

Based on approach from class, adjusted for multiple inputs

This approach was abandoned when team decided to concentrate on CNN model (and various variations).

In [ ]:
import json, os, re, shutil, sys, time
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))
import string          # various string functions
import difflib         # classes and functions for comparing sequences
import utils           # word processing functions and distance functions, pretty printing and data loading

from importlib import reload       # reload external files

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# rnn code
import rnnlstm

# sklearn
import sklearn
from sklearn.model_selection import train_test_split

# Helper libraries
from shared_lib import vocabulary, tf_embed_viz

# set a default vocab size
V=95596
MAX_WORDS = 200000
EMBED_DIM = 300
MAX_SEQUENCE = 25

import plotly.plotly
import plotly.graph_objs as go

# need to look into plotly issues with this line in python3
# plotly.offline.init_notebook_mode() # run at the start of every ipython notebook

from sklearn.metrics import log_loss    # used in measurement / scoring
from sklearn.metrics import classification_report   # among other things, provides accuracy and f1

# Your code
# import rnnlm
# import rnnlm_test
# reload(rnnlm)
# reload(rnnlm_test)

In [ ]:
# Generate training data dataframe
# train_lite for testing approach
# train = pd.read_csv('Data/train_lite.csv') #index_col=0

# full train for when working
train = pd.read_csv('Data/train.csv') #index_col=0

print (len(train))


In [ ]:
def build_vocab(questions, V=10000):
    #token_feed = (canonicalize_word(w) for w in sentence.split() for sentence in questions)
    #token_feed = (canonicalize_word(w) for sentence in questions for w in sentence.split())
    #print (token_feed)
    
    token_feed = []
    for sentence in questions:
        for w in str(sentence).split():
            token_feed.append(utils.canonicalize_word(w))
    token_feed = set(token_feed)
    
    vocab = vocabulary.Vocabulary(token_feed, size=V)
    
    return vocab

In [ ]:
reload(vocabulary)
reload(utils)
question1 = train.question1.tolist()
question2 = train.question2.tolist()
is_duplicate = train.is_duplicate.tolist()

# print('Question pairs: %d' % len(question1))

# build word index
questions = question1 + question2

vocab = build_vocab(questions, V)

In [ ]:
# print(vocab.ordered_words())

In [ ]:
word_index = vocab.ordered_words()

print("Words in index: %d" % len(word_index))

In [ ]:
word_index[0]

In [ ]:
vocab.word_to_id.get('<unk>')

In [ ]:
# critical definition
# maps all words to ID

wordset =  vocab.word_to_id
print (len(wordset))
print (type(wordset))
print (wordset.get('pay'))

In [ ]:
def pad_sent_old(q, max_time):
    if len(q) > max_time:
        # cut it off
        q = q[0:max_time]
    else:
        # pad to max_time
        amt_to_pad = max_time - len(q)
        q.extend([2] * amt_to_pad)
    return q
    

In [ ]:
# pad sentences to make equivalent length
def pad_sent(q):
    if len(q) > MAX_SEQUENCE:
        # cut it off
        q = q[0:MAX_SEQUENCE]
    else:
        # pad to max_time
        amt_to_pad = MAX_SEQUENCE - len(q)
        q.extend([2] * amt_to_pad)
    return q

In [ ]:
def process_questions(questions, wordset, vocab):
    sents = ["<s>" + str(s) + "</s>" for s in questions]
    # print (sents[0:5])
 
    sequence = []
    num_sequence = []
    for s in str(sents):
        # for each sentence in list of sentences
        # print (s)

        s_words = []
        s_w_ids = []
        counter = 0
        for w in str(s).split():
            # capture list of words for sentence
            # once all captured, add to list with sequence
            
            # for each word in split sentence
            # print (w)
            
            # only need to do this for max_sequence counts
            if counter < MAX_SEQUENCE:
                w = w.lower()
                if (wordset == None) or (w in wordset): 
                    s_words.append(w)
                    s_w_ids.append(vocab.word_to_id.get(w))
                else:
                    s_words.append("<unk>") # unknown token
                    s_w_ids.append(vocab.word_to_id.get("<unk>"))
                               
                '''
                if w.isdigit():
                    if (wordset != None) and (w in wordset):
                        q1_sequence.append(w)

                    w = utils.canonicalize_digits(w) # try to canonicalize numbers
                '''
                counter += 1


        # pad or cut to 25 words:
        s_words = pad_sent([s_words])
        s_w_ids = pad_sent([s_w_ids])
        
        sequence.append(s_words)
        num_sequence.append(s_w_ids)
        
        
    sequence = np.array(sequence)
    num_sequence = np.array(num_sequence)
    return sequence, num_sequence

In [ ]:
q1_sequences, q1_seq_num = process_questions(question1, wordset, vocab)
q2_sequences, q2_seq_num = process_questions(question2, wordset, vocab)

In [ ]:
print (q1_sequences.shape)
print (q2_sequences.shape)

# NOTE, with sequences, each original sentence is a list within the list

In [ ]:
reload (rnnlstm)

# building graph based on a3 assigment
TF_GRAPHDIR = "tf_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

# V, H, and num_layers are all things we can play with...
# default corpus is 'brown'
# for now, want to see running with this data
# set hyperparameters
lm = rnnlstm.RNNLM(V=V, H=200, num_layers=2)

lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()
summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)

In [ ]:
#X = np.stack((q1_sequences, q2_sequences), axis=1)
X = np.stack((q1_seq_num, q2_seq_num), axis=1)

#y = is_duplicate
y = np.array(is_duplicate)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

In [ ]:
# max_time = 100

# not sure these matter any more:
max_time = 50
# sentences with > 50 words will be truncated to 50 words
# rest are padded to 50

# for our purposes, a batch is a question pair
# this varies whether train or test though...
# something to think about
#batch_size = len(Q1_train)
batch_size = 1
print (batch_size)


In [ ]:
def batch_generator(q1, q2, label, max_time):
    for i in range(len(q1)):
        # # of q1 and q2 questions is the same, so can use one to iterate
        
        # padded or trunctating all sentences to max_time
        q1[i] = pad_sent(q1[i], max_time)
        q2[i] = pad_sent(q2[i], max_time)
        
    # now we will make them 1 list (vs 2)
    q1 = np.array([idx for sublist in q1 for idx in sublist] )
    q2 = np.array([idx for sublist in q2 for idx in sublist] )
    
    # print (type(q1))
    # print (q1.shape)
        
    q1 = q1.reshape([batch_size,-1])
    q2 = q2.reshape([batch_size,-1])
    label = label.reshape([batch_size, -1])
    
    print (q1.shape)
    print (q1.shape[1])
    
    # need to look at label, why is it 9000???
    
    print (label.shape)
    
    # Yield batches
    # yield q1[i], q2[i], label[i]
    # return q1, q2, label

    # Yield batches
    for i in range(0, q1.shape[1], max_time):
        yield q1[:,i:i+max_time], q2[:,i:i+max_time], label[:,i:i+max_time]

In [ ]:
# possible add another for loop, that allows you to move a sentence over multiple
# need to look at what batch_generator was providing in a3 (perhaps on the cloud even)

In [ ]:
# testing batch generator
bi = batch_generator(Q1_train, Q2_train, y_train, max_time)


for i, (w1, w2, y) in enumerate(bi):
    print ("i:", i)
    print ("w1:", w1)
    print ("w2:", w2)
    print ("len(w1):", len(w1))
    print ("y:", y)

In [ ]:
tt = [[[2,3,4,5]], [[4,5,6,7]], [[2,2,2,2]]]
npt = np.asarray(tt)

In [ ]:
# Training parameters


#######

learning_rate = 0.5
num_epochs = 10

# Model parameters
model_params = dict(V=vocab.size, 
                    H=100, 
                    softmax_ns=200,
                    num_layers=1)

TF_SAVEDIR = "tf_saved"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlstm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlstm_trained")

In [ ]:
q1 = [2, 4660, 2, 555, 7902, 2, 2, 1804, 1908, 10320, 283, 2, 4842, 2]
#q1.reshape([batch_size,-1])
for i in range(0, len(q1), max_time):
    print (i)
    print (type(q1))
    print (type(max_time))
    #print(q1[:,i:i+max_time])

In [ ]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=0.1):
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation
        
    # How do I bring in both w's? Getting a "need to pass W_1 through cost, _, h" function. No idea.

    for i, (w1, w2, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean initial state.
        
        
        

        if i == 0:
            #print ("lm.initial_h_: ", lm.initial_h_)
            #print ("here")
            h = session.run(lm.initial_h_, {lm.input_w_q1_: w1, lm.input_w_q2_: w2})
            #print ("here1")
            

        
        feed_dict = {lm.input_w_q1_: w1, lm.input_w_q2_: w2, lm.target_y_: y, lm.initial_h_: h}
        #print ("w1 shape: ", np.shape(w1))
        #print ("w2 shape: ", np.shape(w2))
        #print ("y shape: ", np.shape(y))
        #print ("h shape: ", np.shape(h))
        
        #print("lm.train_loss_:", lm.train_loss_)
        
        #cost, _, h = session.run([lm.train_loss_, lm.train_step_, lm.final_h_], feed_dict=feed_dict)
        
        train_step, cost, h = session.run([train_op, loss, lm.final_h_],feed_dict=feed_dict)
        
        

        total_cost += cost
        total_batches = i + 1
        total_words += batch_size * max_time

        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print ("[batch %d]: seen %d words at %d wps, loss = %.3f" %
                i, total_words, avg_wps, avg_cost)
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [ ]:
# passing both questions at once
reload(utils)
def score_dataset(lm, session, Q1, Q2, y, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    # bi = utils.batch_generator(Q1, Q2, y, batch_size=50, max_time=100)
    bi = batch_generator(Q1, Q2, y, max_time)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=1.0, train=False, 
                     verbose=False, tick_s=3600)
    print ("%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost)))

In [ ]:
reload (rnnlstm)
reload (utils)

# Will print status every this many seconds
print_interval = 5

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

lm = rnnlstm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        #bi = utils.batch_generator(Q1_train, Q2_train, y_train, batch_size, max_time)
        bi = batch_generator(Q1_train, Q2_train, y_train, max_time)
        print ("[epoch %d] Starting epoch %d" % (epoch, epoch))

        # Run a training epoch.
        
        run_epoch(lm, session, bi, train=True) 

        print ("[epoch %d] Completed in %s" % (epoch, utils.pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print ("[epoch %d]" % epoch),
        #score_dataset(lm, session, train_ids, name="Train set")
        print ("[epoch %d]" % epoch),
        score_dataset(lm, session, Q1_test, Q2_test, y_test, name="Test set")
        print ("")
    
    # Save final model
    saver.save(session, trained_filename)